# Optimal climate policy via Dyanmic Programming (ocpvdp)

by John M. Deutch, Henri Drake, Alan Edelman, and Ronald L. Rivest.

This notebook explores the use of dynamic programming methods 
(using discrete Markov Decision Processes) to find optimal climate change strategies.
Reinforcement learning methods should also be applicable.  

We adopt the goal of sticking within a given temperature target, and minimizing 
(discounted) cost to do so.

This formulation allows for various stochastic transitions (as is standard in the
Markov Decision Processes).

A state is defined by a triple
    (t, c, T) = (time, CO2-concentration, temperature).
(In some formulations, the temperature may be derived from the CO2 concentation,
but we allow the more general formulation here, so that temperature equilibration
may take some time.)

Components of a state are discretized: time in years, CO2 concentration in ppmv, and
temperature in units of 0.01 degrees K.  The state components are thus all integers.
Moreover, each component is quantized, to reduce total number of states.

The available actions are 
* R (CO2 reduction), 
* N (Negative emissions), 
* A (Adaptation), and 
* G (geoengineering),

or any combination thereof.  More precisely, these actions are discretized in units of 
0.05 between 0 and 1, so there are $4^{21}$ possible actions from any state.

Costs and damages are discounted by a discount rate $\gamma$.


In [1]:
# Definition of a model state

struct state                 # Note that a state is non-mutable
    qt::UInt16               # (quantized) time since start
    qc::UInt16               # (quantized) CO2 concentration total (nonnegative)
    qT::Int16                # (quantized) change in temperature since pre-industrial
end

In [2]:
using Formatting

## Quantization

Values in a state are biased and quantized, to make number of states manageable.
The following cells give the bias (offset) and bin size for each of the 
three state components.

In [3]:
# Time quantization
# Note that simulation steps have size equal to bin_size_t

const offset_t = 2020
const bin_size_t = 1

# Given a time t in years, return quantized version, as used in state
function quantize_t(t)
    qt = UInt16(round((t-offset_t)/bin_size_t))
    return qt
end

function inv_quantize_t(qt)
    t = offset_t + bin_size_t * qt
    return t
end

function test_quantize_t()
    println("quantize_t(2030) = $(quantize_t(2030))")
    println("inv_quantize_t(10) = $(inv_quantize_t(10))")
end

test_quantize_t()

quantize_t(2030) = 10
inv_quantize_t(10) = 2030


In [4]:
# CO2 quantization

const offset_c = 0               # in ppmv
const bin_size_c = 40            # in ppmv
const pre_c = 280                # in ppmv

function quantize_c(c::Real)
    # println("quantize_c(c=$c)")
    c = max(c, 0)
    qc = UInt16(round((c-offset_c)/bin_size_c))
    return qc
end

function inv_quantize_c(qc)
    c = offset_c + bin_size_c * qc
    return c
end

function test_quantize_c()
    qc456 = quantize_c(456)
    iqc456 = inv_quantize_c(qc456)
    println("quantize_c(456) = $qc456")
    println("inv_quantize_c($qc456) = $iqc456")
end

test_quantize_c()

quantize_c(456) = 11
inv_quantize_c(11) = 440


In [5]:
# Temperature quantization
# Relevant temperature rise is that since pre-industrial times
# (Or since start of simulation?  Make consistent...)

const offset_T = 288                
const bin_size_T = 0.5
const pre_T = 288                # check

function quantize_T(T)
    qT = Int16(round((T-offset_T)/bin_size_T))
    return qT
end

function inv_quantize_T(qT)
    T = offset_T + bin_size_T * qT
    return T
end

function test_quantize_T()
    qT290 = quantize_T(290.1)
    iqT290 = inv_quantize_T(qT290)
    println("quantize_T(290.1) = $qT290")
    println("inv_quantize_T($qT290) = $iqT290")
end

test_quantize_T()

quantize_T(290.1) = 4
inv_quantize_T(4) = 290.0


In [6]:
# state quantization -- does all of t, c, T quantization

function quantize_tcT(t, c, T)::state
    s = state(quantize_t(t), quantize_c(c), quantize_T(T))
    return s
end

function inv_quantize_tcT(s::state)
    t = inv_quantize_t(s.qt)
    c = inv_quantize_c(s.qc)
    T = inv_quantize_T(s.qT)
    return (t, c, T)
end

function test_quantize_tcT()
    println("quantize_tcT(2030, 500, 292) = $(quantize_tcT(2030, 500, 292))")
    println("inv_quantize_tcT(state(0x000a, 0x0009, 0x0028)) = $(inv_quantize_tcT(state(0x000a, 0x0009, 0x0028)))")
end

test_quantize_tcT()

s0 = quantize_tcT(2020, 400, 288)

quantize_tcT(2030, 500, 292) = state(0x000a, 0x000c, 8)
inv_quantize_tcT(state(0x000a, 0x0009, 0x0028)) = (2030, 360, 308.0)


state(0x0000, 0x000a, 0)

In [7]:
function print_state(s::state)
    print(inv_quantize_tcT(s))
end

function test_print_state()
    print_state(s0)
end

test_print_state()

(2020, 400, 288.0)

## Probabilistic mapping of triple to states

A given triple of values is mapped to a list of four states, with their
associated probabilities.  This has the flavor of "randomized rounding", but
since we are in a framework where actions can have probabilistic consequences,
we don't actually need to pick just one, and can work with all (four) possible
"rounded" values.

In [8]:
function states_for(t, c, T)
    qt = quantize_t(t)
    
    qcs = []
    qc = quantize_c(c) 
    iqc = inv_quantize_c(qc)
    if isapprox(c, iqc)
        push!(qcs, (qc, 1.0))
    elseif c < iqc
        c0 = iqc - bin_size_c
        qc0 = qc - 1
        c1 = iqc
        qc1 = qc
        push!(qcs, (qc0, (c1-c)/(c1-c0)))
        push!(qcs, (qc1, (c-c0)/(c1-c0)))
    elseif c > iqc
        c0 = iqc
        qc0 = qc
        c1 = iqc + bin_size_c
        qc1 = qc + 1
        push!(qcs, (qc0, (c1-c)/(c1-c0)))
        push!(qcs, (qc1, (c-c0)/(c1-c0)))
    end
    
    qTs = []
    qT = quantize_T(T)
    iqT = inv_quantize_T(qT)
    if isapprox(T, iqT)
        push!(qTs, (qT, 1.0))
    elseif T < iqT
        T0 = iqT - bin_size_T
        qT0 = qT-1
        T1 = iqT
        qT1 = qT
        push!(qTs, (qT0, (T1-T)/(T1-T0)))
        push!(qTs, (qT1, (T-T0)/(T1-T0)))
    elseif T > iqT
        T0 = iqT
        qT0 = qT
        T1 = iqT + bin_size_T
        qT1 = qT+1
        push!(qTs, (qT0, (T1-T)/(T1-T0)))
        push!(qTs, (qT1, (T-T0)/(T1-T0)))
    end

    L = []
    for (qc, pc) in qcs
        for (qT, pT) in qTs
            push!(L, (state(qt, qc, qT), pc*pT))
        end
    end
    return L
end


function test_states_for_single(t, c, T)
    println("test_states_for_single")
    println("  Input:")
    println("    ($t, $c, $T)")
    println("  Output:")
    for (s, p) in states_for(t, c, T)
        println(format("    {} with prob {:0.3f}", inv_quantize_tcT(s), p))
    end
end

function test_states_for()
    test_states_for_single(2022, 410, 288.2)
    test_states_for_single(2022, 410, 288.0)
    test_states_for_single(2023, 400, 288.2)
    test_states_for_single(2024, 400, 288.0)
end

test_states_for()

test_states_for_single
  Input:
    (2022, 410, 288.2)
  Output:
    (2022, 400, 288.0) with prob 0.450
    (2022, 400, 288.5) with prob 0.300
    (2022, 440, 288.0) with prob 0.150
    (2022, 440, 288.5) with prob 0.100
test_states_for_single
  Input:
    (2022, 410, 288.0)
  Output:
    (2022, 400, 288.0) with prob 0.750
    (2022, 440, 288.0) with prob 0.250
test_states_for_single
  Input:
    (2023, 400, 288.2)
  Output:
    (2023, 400, 288.0) with prob 0.600
    (2023, 400, 288.5) with prob 0.400
test_states_for_single
  Input:
    (2024, 400, 288.0)
  Output:
    (2024, 400, 288.0) with prob 1.000


# Temperature

In [9]:
# Equilibrium Temperature Rise

const ϵ = 3 / log(2)

function equilibrium_dT(c)
    # assume a doubling of co2 gives a 3 degree C rise in temp since pre-industrial
    dT = ϵ * log(c / pre_c)
    return dT
end

function equilibrium_T(c)
    # return equilibrium temperature for a given co2 level
    T = pre_T + equilibrium_dT(c)
    return T
end

function test_equilibrium_dT()
    println("Equilibrium temperature rise from pre for a co2 concentration of 560 ppmv is $(equilibrium_dT(560))")
    println("Equilibrium temperature for a co2 concentration of 560 ppmv is $(equilibrium_T(560))")
end

test_equilibrium_dT()

Equilibrium temperature rise from pre for a co2 concentration of 560 ppmv is 3.0
Equilibrium temperature for a co2 concentration of 560 ppmv is 291.0


In [10]:
# convergence to equilibrium temperature
# assumes exponential converge 

T_halving_time = 50    # years until distance to equilibrium temperature is halved
                       # what should this value be?

function dynamic_dT(T, c)
    # current temperature is T
    # current co2 level is c
    # return what temperature change will be over next year
    eq_T = equilibrium_T(c)
    dT = (eq_T - T) * (1 - exp(log(0.5)/T_halving_time))
    return dT
end

function test_dynamic_dT()
    dT = dynamic_dT(288, 560)
    println("dyanmic_dT for T=288 and c=560 is $dT")
    println("I.e., temperature will rise by $dT degrees K in the next year.")
end

test_dynamic_dT()

dyanmic_dT for T=288 and c=560 is 0.04130188651992239
I.e., temperature will rise by 0.04130188651992239 degrees K in the next year.


In [11]:
# define action

# Each action has four components, specified by floats between
# 0 and 1, inclusive.
#    R      fraction of current emissions reduced/eliminated
#    N      fraction of current concentration removed from atmosphere
#    A      fraction of GWP devoted to adaptation
#    G      fraction reduction in solar insolation

struct action
    R::Float16           # emissions Reduction
    N::Float16           # Negative emissions
    A::Float16           # Adaptation
    G::Float16           # Geoengineering
end

function print_action(a::action)
    print(format("(R={:4.3f}, N={:4.3f}, A={:4.3f}, G={:4.3f})", a.R, a.N, a.A, a.G))
end

print_action(action(0.1, 0.2, 0.3, 0.4))

(R=0.100, N=0.200, A=0.300, G=0.400)

# Actions

In [12]:
function generate_actions(s::state)
    # generate actions possible for the given state
    # this is just a first-cut way of generating a list
    # should it depend on the state?
    L = []
    for R in [0.1, 0.4, 0.7]
        for N in [0.001, 0.002, 0.005, 0.010]
            for A in [0.01, 0.03, 0.07]
                for G in [0.02, 0.05, 0.08]
                    push!(L, action(R, N, A, G))
                end
            end
        end
    end
    return L
end

function test_generate_actions()
    as = generate_actions(s0)
    print("For state ")
    print_state(s0)
    println()
    println("The $(length(as)) actions possible are:")
    for a in generate_actions(s0)
        print_action(a)
        println()
    end
end

test_generate_actions()

For state (2020, 400, 288.0)
The 108 actions possible are:
(R=0.100, N=0.001, A=0.010, G=0.020)
(R=0.100, N=0.001, A=0.010, G=0.050)
(R=0.100, N=0.001, A=0.010, G=0.080)
(R=0.100, N=0.001, A=0.030, G=0.020)
(R=0.100, N=0.001, A=0.030, G=0.050)
(R=0.100, N=0.001, A=0.030, G=0.080)
(R=0.100, N=0.001, A=0.070, G=0.020)
(R=0.100, N=0.001, A=0.070, G=0.050)
(R=0.100, N=0.001, A=0.070, G=0.080)
(R=0.100, N=0.002, A=0.010, G=0.020)
(R=0.100, N=0.002, A=0.010, G=0.050)
(R=0.100, N=0.002, A=0.010, G=0.080)
(R=0.100, N=0.002, A=0.030, G=0.020)
(R=0.100, N=0.002, A=0.030, G=0.050)
(R=0.100, N=0.002, A=0.030, G=0.080)
(R=0.100, N=0.002, A=0.070, G=0.020)
(R=0.100, N=0.002, A=0.070, G=0.050)
(R=0.100, N=0.002, A=0.070, G=0.080)
(R=0.100, N=0.005, A=0.010, G=0.020)
(R=0.100, N=0.005, A=0.010, G=0.050)
(R=0.100, N=0.005, A=0.010, G=0.080)
(R=0.100, N=0.005, A=0.030, G=0.020)
(R=0.100, N=0.005, A=0.030, G=0.050)
(R=0.100, N=0.005, A=0.030, G=0.080)
(R=0.100, N=0.005, A=0.070, G=0.020)
(R=0.100, N=0.00

In [13]:
const CR = 50.0   / 80.0  # trillion $  (from JMD's paper)
const CN = 100.0  / 80.0  # trillion $
const CA = 150.0  / 80.0  # trillion $
const CG = 150.0  / 80.0  # trillion $

damage_limit_c = 560               # get large penalty if exceed this c threshold
damage_limit_c_penalty = 45e12     # this is the large penalty

β = 2.22e9 / 80.0  # dollars per (degree C)^2   # from JMD's paper, divided by 80
# needs to be fixed to assume growing economy

function action_cost(s::state, a::action)
    # return cost (including damages) of executing given action in given state
    t, c, T = inv_quantize_tcT(s)
    R, N, A, G = (a.R, a.N, a.A, a.G)
    # cost of executing action
    # does not (yet) have Deutch's nonlinear functions included
    cost = a.R * CR  +  a.N * CN  +  a.A * CA  +  a.G * CG
    # plus damages acrued
    if c > damage_limit_c
        damages = damage_limit_c_penalty
    else
        damages = β * (1-A) * (T-pre_T)^2 * (1-G)^2
    end
    cost += damages
    return cost
end

function test_action_cost()
    println("test_action_cost")
    println("Input:")
    print("  start state is "); print_state(s0); println()
    println("Output:")
    for a in generate_actions(s0)
        R, N, A, G = a.R, a.N, a.A, a.G
        print(format("  action "))
        print_action(a)
        println(format(" costs {:4.3f} \$T", action_cost(s0, a)))
    end
end

test_action_cost()

test_action_cost
Input:
  start state is (2020, 400, 288.0)
Output:
  action (R=0.100, N=0.001, A=0.010, G=0.020) costs 0.120 $T
  action (R=0.100, N=0.001, A=0.010, G=0.050) costs 0.176 $T
  action (R=0.100, N=0.001, A=0.010, G=0.080) costs 0.233 $T
  action (R=0.100, N=0.001, A=0.030, G=0.020) costs 0.157 $T
  action (R=0.100, N=0.001, A=0.030, G=0.050) costs 0.214 $T
  action (R=0.100, N=0.001, A=0.030, G=0.080) costs 0.270 $T
  action (R=0.100, N=0.001, A=0.070, G=0.020) costs 0.233 $T
  action (R=0.100, N=0.001, A=0.070, G=0.050) costs 0.289 $T
  action (R=0.100, N=0.001, A=0.070, G=0.080) costs 0.345 $T
  action (R=0.100, N=0.002, A=0.010, G=0.020) costs 0.121 $T
  action (R=0.100, N=0.002, A=0.010, G=0.050) costs 0.177 $T
  action (R=0.100, N=0.002, A=0.010, G=0.080) costs 0.234 $T
  action (R=0.100, N=0.002, A=0.030, G=0.020) costs 0.159 $T
  action (R=0.100, N=0.002, A=0.030, G=0.050) costs 0.215 $T
  action (R=0.100, N=0.002, A=0.030, G=0.080) costs 0.271 $T
  action (R=0.100

In [14]:
const annual_increment_c = 2 # ppmv  (should grow with economy!)

function apply_action(s::state, a::action)
    # return list of states that may result from action, with probabilities.
    # consequences are deterministic in model, but because of quantization, we
    # may "round" result in one of four ways (rounding c and/or T) to get new state(s).
    t, c, T = inv_quantize_tcT(s)
    # first add generic addition of co2 due to economic activity (2 ppmv), minus reduction
    dc = annual_increment_c
    c += dc * (1.0 - a.R)
    # apply negative emissions  (fraction a.N removed from atmosphere)
    c *= (1 - a.N)    
    # estimate new temperature
    T2 = T + dynamic_dT(T, c)
    return states_for(t+1, c, T2)
end

function test_apply_action(s, a)
    println("test_apply_action")
    println("Input:")
    print("  state "); print_state(s); println()
    print("  action"); print_action(a); println()
    println("Output (probabilistic transition):")
    for (s2, p) in apply_action(s, a)
        print("  ")
        print_state(s2)
        println(" p = $p")
    end
end

test_apply_action(s0, action(0.2, 0.002, 0.35, 0.04))

test_apply_action
Input:
  state (2020, 400, 288.0)
  action(R=0.200, N=0.002, A=0.350, G=0.040)
Output (probabilistic transition):
  (2021, 400, 288.0) p = 0.9377302440563193
  (2021, 400, 288.5) p = 0.04187669430580225
  (2021, 440, 288.0) p = 0.0195212895273249
  (2021, 440, 288.5) p = 0.0008717721105535186


# Graph model

In [15]:
# routines to enumerate all (successor) states in the model

function enumerate_successors_of_state(s::state)
    as = generate_actions(s)
    successor_states = Set()
    for a in as
        for (s, p) in apply_action(s, a)
            push!(successor_states, s)
        end
    end
    L = collect(successor_states)
    sort!(L, by = s -> (s.qt, s.qc, s.qT))
    return L
end


function test_enumerate_successors_of_state(s::state)
    println("test_enumerate_successors_of_state ")
    println("Input:")
    print("  "); print_state(s); println()
    successor_states = enumerate_successors_of_state(s0)
    println("Output:")
    for s in successor_states
        print("  "); print_state(s); println()
    end
end

test_enumerate_successors_of_state(s0)

test_enumerate_successors_of_state 
Input:
  (2020, 400, 288.0)
Output:
  (2021, 360, 288.0)
  (2021, 360, 288.5)
  (2021, 400, 288.0)
  (2021, 400, 288.5)
  (2021, 440, 288.0)
  (2021, 440, 288.5)


In [16]:
function enumerate_successors_of_state_set(S)
    successor_states = Set()
    for s in S
        for s2 in enumerate_successors_of_state(s)
            push!(successor_states, s2)
        end
    end
    L = collect(successor_states)
    sort!(L, by = s -> (s.qt, s.qc, s.qT))
    return L
end

function test_enumerate_successors_of_state_set(S)
    println("test_enumerate_successors_of_state_set")
    println("Input:")
    for s in S
        print("  "); print_state(s); println()
    end
    println("Output:")
    successor_states = enumerate_successors_of_state_set(S)
    for s in successor_states
        print("  "); print_state(s); println()
    end
end

test_enumerate_successors_of_state_set(enumerate_successors_of_state(s0))

test_enumerate_successors_of_state_set
Input:
  (2021, 360, 288.0)
  (2021, 360, 288.5)
  (2021, 400, 288.0)
  (2021, 400, 288.5)
  (2021, 440, 288.0)
  (2021, 440, 288.5)
Output:
  (2022, 320, 288.0)
  (2022, 320, 288.5)
  (2022, 320, 289.0)
  (2022, 360, 288.0)
  (2022, 360, 288.5)
  (2022, 360, 289.0)
  (2022, 400, 288.0)
  (2022, 400, 288.5)
  (2022, 400, 289.0)
  (2022, 440, 288.0)
  (2022, 440, 288.5)
  (2022, 440, 289.0)
  (2022, 480, 288.0)
  (2022, 480, 288.5)
  (2022, 480, 289.0)


In [17]:
function enumerate_states_by_year(s0, H)
    S0 = Set([s0])
    first_year = inv_quantize_t(s0.qt)
    states_by_year = Dict()
    states_by_year[first_year] = S0
    for year in first_year+1:H
        states_by_year[year] = enumerate_successors_of_state_set(states_by_year[year-1])
    end
    return states_by_year
end

function test_enumerate_states_by_year(s0, H)
    println("test_enumerate_states_by_year")
    print("  Starting with state "); print_state(s0); println()
    println("  Running until year H = ", H)
    states_by_year = enumerate_states_by_year(s0, H)
    println("  Number of states by year in output:")
    num_states_total = 0
    for year in inv_quantize_t(s0.qt):H
        num_in_year = length(states_by_year[year])
        num_states_total += num_in_year
        println("    ", year, ": ", num_in_year)
    end
    println("    Total number of states: $num_states_total")
end

test_enumerate_states_by_year(s0, 2100)


test_enumerate_states_by_year
  Starting with state (2020, 400, 288.0)
  Running until year H = 2100
  Number of states by year in output:
    2020: 1
    2021: 6
    2022: 15
    2023: 27
    2024: 43
    2025: 63
    2026: 87
    2027: 114
    2028: 146
    2029: 181
    2030: 208
    2031: 238
    2032: 267
    2033: 296
    2034: 329
    2035: 361
    2036: 396
    2037: 431
    2038: 466
    2039: 504
    2040: 543
    2041: 580
    2042: 617
    2043: 656
    2044: 694
    2045: 732
    2046: 769
    2047: 809
    2048: 848
    2049: 887
    2050: 925
    2051: 966
    2052: 1006
    2053: 1046
    2054: 1086
    2055: 1126
    2056: 1168
    2057: 1209
    2058: 1232
    2059: 1255
    2060: 1277
    2061: 1299
    2062: 1321
    2063: 1343
    2064: 1365
    2065: 1386
    2066: 1406
    2067: 1425
    2068: 1443
    2069: 1461
    2070: 1478
    2071: 1494
    2072: 1510
    2073: 1525
    2074: 1538
    2075: 1550
    2076: 1562
    2077: 1573
    2078: 1583
    2079: 1592
  

# Economy

In [18]:
# Definition of growth and discount rates; time-related constants

const years_per_step = 1  # years per simulation step (unused; assumed 1 in code)

const g = 0.02            # growth rate of economy, per year

const γ = 0.02            # discount rate for damages, per year
                          # also discount rate for budget expenditures



0.02

# Optimal policy

In [25]:
function initial_value_of_state(s)
    # give initial value of state s
    t, c, T = inv_quantize_tcT(s)
    GWP_2020 = 90e12                     # dollars, from JMD paper
    GWP_t = GWP_2020 * (1+g)^(t-2020)
    v = GWP_t
    if c > damage_limit_c
        v -= damage_limit_c_penalty
    else
        v -= β * (T-pre_T)^2
    end
    return v
end

function initial_value_of_states(S)
    V = Dict()
    for s in S
        V[s] = initial_value_of_state(s)
    end
    return V
end    

function optimize_policy(s0::state, H)
    # find optimal policy for given start state, up to year H
    # assumes optimal policy for successor states already found

    first_year = inv_quantize_t(s0.qt)
    last_year = H
    states_by_year = enumerate_states_by_year(s0, last_year)

    V = initial_value_of_states(states_by_year[last_year])
    policy = Dict()
    
    for year in last_year-1:-1:first_year
        for s in states_by_year[year]
            s_done = false
            actions = generate_actions(s)
            max_value_a = -1e80
            for a in actions
                value_a = -action_cost(s, a)
                for (s2, p) in apply_action(s, a)
                    value_a += V[s2] * p * (1-γ)
                end
                if value_a > max_value_a || max_value_a == -1e80
                    max_value_a = value_a
                    policy[s] = a
                    V[s] = value_a
                    s_done = true
                end
            end
            if !s_done
                print("state not done: "); print_state(s); println()
            end
        end
    end
    return states_by_year, V, policy
end    
        
function test_optimize_policy(last_year)
    println("test_optimize_policy")
    println("  For each state s, give optimal policy action and value V of state s (more value is better)")
    first_year = inv_quantize_t(s0.qt)
    states_by_year, V, policy = optimize_policy(s0, last_year)
    for year in first_year:last_year
        for s in states_by_year[year]
            print("  ")
            print_state(s); print(" ") 
            print_action(policy[s]); print(" ")
            println("V = $(V[s])")
        end
    end
end

test_optimize_policy(2100)

test_optimize_policy
  For each state s, give optimal policy action and value V of state s (more value is better)
  (2020, 400, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) V = 8.716489255378753e13
  (2021, 360, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) V = 8.89438407356582e13
  (2021, 360, 288.5) (R=0.700, N=0.010, A=0.070, G=0.080) V = 8.894366000268672e13
  (2021, 400, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) V = 8.894377144476366e13
  (2021, 400, 288.5) (R=0.700, N=0.010, A=0.070, G=0.080) V = 8.89435442842933e13
  (2021, 440, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) V = 8.894368027780453e13
  (2021, 440, 288.5) (R=0.700, N=0.010, A=0.070, G=0.080) V = 8.894340321556983e13
  (2022, 320, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) V = 9.075906924651808e13
  (2022, 320, 288.5) (R=0.700, N=0.010, A=0.070, G=0.080) V = 9.075892452708469e13
  (2022, 320, 289.0) (R=0.700, N=0.010, A=0.070, G=0.080) V = 9.075854878252677e13
  (2022, 360, 288.0) (R=0.700, N=0.010, A=0.010, G=0.020) 

Excessive output truncated after 524293 bytes.

 V = 1.3871197791858706e14
  (2043, 320, 293.0) (R=0.700, N=0.010, A=0.070, G=0.080) V = 1.3870987490719117e14
  (2043, 360, 281.5) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3870423298630997e14
  (2043, 360, 282.0) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3870682393129061e14
  (2043, 360, 282.5) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3870919652397267e14
  (2043, 360, 283.0) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871135078528422e14
  (2043, 360, 283.5) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871328675659375e14
  (2043, 360, 284.0) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871500451951472e14
  (2043, 360, 284.5) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871650423502556e14
  (2043, 360, 285.0) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871778622134764e14
  (2043, 360, 285.5) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871885111114038e14
  (2043, 360, 286.0) (R=0.100, N=0.001, A=0.070, G=0.080) V = 1.3871970017640233e14
  (2043, 360, 286.5) (R=0.100, N=0.001, A=0.070, 

KeyError: KeyError: key state(0x0050, 0x0001, -17) not found